<a href="https://colab.research.google.com/github/Dusein/MachineLearningTask/blob/main/10thWeekTask/Regression_Model_MLP_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Import library dan setup dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Ganti 'your_file_path' dengan path sebenarnya ke file CSV Anda di Google Drive
file_path = '/content/drive/My Drive/Dataset/ENB2012_data.xlsx'

# Membaca file CSV ke dalam DataFrame
data = pd.read_excel(file_path)

# Menampilkan beberapa baris pertama dari DataFrame
print(data.head())

     X1     X2     X3      X4   X5  X6   X7  X8     Y1     Y2
0  0.98  514.5  294.0  110.25  7.0   2  0.0   0  15.55  21.33
1  0.98  514.5  294.0  110.25  7.0   3  0.0   0  15.55  21.33
2  0.98  514.5  294.0  110.25  7.0   4  0.0   0  15.55  21.33
3  0.98  514.5  294.0  110.25  7.0   5  0.0   0  15.55  21.33
4  0.90  563.5  318.5  122.50  7.0   2  0.0   0  20.84  28.28


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Separate features (X) and target (Y)
X = data.iloc[:, :-2].values  # X1 to X8
Y = data.iloc[:, -2:].values  # Y1 and Y2

# Normalize features using MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

# Display the shapes of the splits
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

# Konversi data ke format PyTorch
X_train_tensor = torch.FloatTensor(X_train)
Y_train_tensor = torch.FloatTensor(Y_train)
X_test_tensor = torch.FloatTensor(X_test)
Y_test_tensor = torch.FloatTensor(Y_test)

# Dataset PyTorch
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)


In [15]:
# Definisi MLP Regression
class MLPRegression(nn.Module):
    def __init__(self, input_size, hidden_layers, neurons, activation_fn):
        super(MLPRegression, self).__init__()
        layers = []
        current_size = input_size

        for _ in range(hidden_layers):
            layers.append(nn.Linear(current_size, neurons))
            layers.append(activation_fn)
            current_size = neurons

        layers.append(nn.Linear(current_size, 2))  # Output size = 2
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


In [12]:
# Parameter eksperimen
hidden_layer_options = [1, 2, 3]

# Eksperimen Hidden Layers
results_hidden_layers = []
criterion = nn.MSELoss()

for hidden_layers in hidden_layer_options:
    # Inisialisasi model
    model = MLPRegression(input_size=8, hidden_layers=hidden_layers, neurons=8, activation_fn=nn.ReLU())
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Latihan
    model.train()
    for epoch in range(50):
        for batch_X, batch_Y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_Y)
            loss.backward()
            optimizer.step()

    # Evaluasi
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor)
        test_loss = criterion(predictions, Y_test_tensor).item()
        results_hidden_layers.append({"hidden_layers": hidden_layers, "test_loss": test_loss})

# Cetak hasil
print("Hasil eksperimen jumlah hidden layers:")
for result in results_hidden_layers:
    print(result)


Hasil eksperimen jumlah hidden layers:
{'hidden_layers': 1, 'test_loss': 40.264373779296875}
{'hidden_layers': 2, 'test_loss': 18.112119674682617}
{'hidden_layers': 3, 'test_loss': 14.262744903564453}


In [13]:
# Parameter eksperimen
neuron_options = [4, 8, 16, 32, 64]

# Eksperimen Jumlah Neuron
results_neurons = []

for neurons in neuron_options:
    model = MLPRegression(input_size=8, hidden_layers=2, neurons=neurons, activation_fn=nn.ReLU())
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Latihan
    model.train()
    for epoch in range(50):
        for batch_X, batch_Y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_Y)
            loss.backward()
            optimizer.step()

    # Evaluasi
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor)
        test_loss = criterion(predictions, Y_test_tensor).item()
        results_neurons.append({"neurons": neurons, "test_loss": test_loss})

# Cetak hasil
print("Hasil eksperimen jumlah neurons:")
for result in results_neurons:
    print(result)


Hasil eksperimen jumlah neurons:
{'neurons': 4, 'test_loss': 630.0354614257812}
{'neurons': 8, 'test_loss': 13.341109275817871}
{'neurons': 16, 'test_loss': 12.234487533569336}
{'neurons': 32, 'test_loss': 11.397029876708984}
{'neurons': 64, 'test_loss': 10.528437614440918}


In [14]:
# Parameter eksperimen
activation_functions = {
    "linear": nn.Identity(),
    "sigmoid": nn.Sigmoid(),
    "relu": nn.ReLU(),
    "softmax": nn.Softmax(dim=1),
    "tanh": nn.Tanh()
}

# Eksperimen Fungsi Aktivasi
results_activation = []

for activation_name, activation_fn in activation_functions.items():
    model = MLPRegression(input_size=8, hidden_layers=2, neurons=8, activation_fn=activation_fn)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Latihan
    model.train()
    for epoch in range(50):
        for batch_X, batch_Y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_Y)
            loss.backward()
            optimizer.step()

    # Evaluasi
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor)
        test_loss = criterion(predictions, Y_test_tensor).item()
        results_activation.append({"activation_function": activation_name, "test_loss": test_loss})

# Cetak hasil
print("Hasil eksperimen fungsi aktivasi:")
for result in results_activation:
    print(result)


Hasil eksperimen fungsi aktivasi:
{'activation_function': 'linear', 'test_loss': 14.724620819091797}
{'activation_function': 'sigmoid', 'test_loss': 312.8377685546875}
{'activation_function': 'relu', 'test_loss': 17.302885055541992}
{'activation_function': 'softmax', 'test_loss': 572.2593994140625}
{'activation_function': 'tanh', 'test_loss': 304.16571044921875}


In [16]:
# Parameter eksperimen
epoch_options = [1, 10, 25, 50, 100, 250]

# Eksperimen Epoch
results_epochs = []

for epochs in epoch_options:
    model = MLPRegression(input_size=8, hidden_layers=2, neurons=8, activation_fn=nn.ReLU())
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Latihan
    model.train()
    for epoch in range(epochs):
        for batch_X, batch_Y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_Y)
            loss.backward()
            optimizer.step()

    # Evaluasi
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor)
        test_loss = criterion(predictions, Y_test_tensor).item()
        results_epochs.append({"epochs": epochs, "test_loss": test_loss})

# Cetak hasil
print("Hasil eksperimen jumlah epochs:")
for result in results_epochs:
    print(result)


Hasil eksperimen jumlah epochs:
{'epochs': 1, 'test_loss': 666.0958862304688}
{'epochs': 10, 'test_loss': 340.1918640136719}
{'epochs': 25, 'test_loss': 56.46638488769531}
{'epochs': 50, 'test_loss': 15.718113899230957}
{'epochs': 100, 'test_loss': 11.590465545654297}
{'epochs': 250, 'test_loss': 11.636409759521484}


In [17]:
# Parameter eksperimen
lr_options = [10, 1, 0.1, 0.01, 0.001, 0.0001]

# Eksperimen Learning Rate
results_lr = []

for lr in lr_options:
    model = MLPRegression(input_size=8, hidden_layers=2, neurons=8, activation_fn=nn.ReLU())
    optimizer = optim.Adam(model.parameters(), lr=lr)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Latihan
    model.train()
    for epoch in range(50):
        for batch_X, batch_Y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_Y)
            loss.backward()
            optimizer.step()

    # Evaluasi
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor)
        test_loss = criterion(predictions, Y_test_tensor).item()
        results_lr.append({"learning_rate": lr, "test_loss": test_loss})

# Cetak hasil
print("Hasil eksperimen learning rate:")
for result in results_lr:
    print(result)


Hasil eksperimen learning rate:
{'learning_rate': 10, 'test_loss': 98.9422836303711}
{'learning_rate': 1, 'test_loss': 99.19324493408203}
{'learning_rate': 0.1, 'test_loss': 9.822626113891602}
{'learning_rate': 0.01, 'test_loss': 9.992133140563965}
{'learning_rate': 0.001, 'test_loss': 12.942505836486816}
{'learning_rate': 0.0001, 'test_loss': 602.1897583007812}


In [18]:
# Parameter eksperimen
batch_size_options = [16, 32, 64, 128, 256, 512]

# Eksperimen Batch Size
results_batch_size = []

for batch_size in batch_size_options:
    model = MLPRegression(input_size=8, hidden_layers=2, neurons=8, activation_fn=nn.ReLU())
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Latihan
    model.train()
    for epoch in range(50):
        for batch_X, batch_Y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_Y)
            loss.backward()
            optimizer.step()

    # Evaluasi
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor)
        test_loss = criterion(predictions, Y_test_tensor).item()
        results_batch_size.append({"batch_size": batch_size, "test_loss": test_loss})

# Cetak hasil
print("Hasil eksperimen batch size:")
for result in results_batch_size:
    print(result)


Hasil eksperimen batch size:
{'batch_size': 16, 'test_loss': 14.060678482055664}
{'batch_size': 32, 'test_loss': 13.798885345458984}
{'batch_size': 64, 'test_loss': 48.44343566894531}
{'batch_size': 128, 'test_loss': 340.23822021484375}
{'batch_size': 256, 'test_loss': 591.6143188476562}
{'batch_size': 512, 'test_loss': 628.1282348632812}


In [23]:
# Cetak hasil
print("Hasil eksperimen jumlah hidden layers:")
for result in results_hidden_layers:
    print(result)

# Cetak hasil
print("\nHasil eksperimen jumlah neurons:")
for result in results_neurons:
    print(result)

# Cetak hasil
print("\nHasil eksperimen fungsi aktivasi:")
for result in results_activation:
    print(result)

# Cetak hasil
print("\nHasil eksperimen jumlah epochs:")
for result in results_epochs:
    print(result)

# Cetak hasil
print("\nHasil eksperimen learning rate:")
for result in results_lr:
    print(result)

# Cetak hasil
print("\nHasil eksperimen batch size:")
for result in results_batch_size:
    print(result)

Hasil eksperimen jumlah hidden layers:
{'hidden_layers': 1, 'test_loss': 40.264373779296875}
{'hidden_layers': 2, 'test_loss': 18.112119674682617}
{'hidden_layers': 3, 'test_loss': 14.262744903564453}

Hasil eksperimen jumlah neurons:
{'neurons': 4, 'test_loss': 630.0354614257812}
{'neurons': 8, 'test_loss': 13.341109275817871}
{'neurons': 16, 'test_loss': 12.234487533569336}
{'neurons': 32, 'test_loss': 11.397029876708984}
{'neurons': 64, 'test_loss': 10.528437614440918}

Hasil eksperimen fungsi aktivasi:
{'activation_function': 'linear', 'test_loss': 14.724620819091797}
{'activation_function': 'sigmoid', 'test_loss': 312.8377685546875}
{'activation_function': 'relu', 'test_loss': 17.302885055541992}
{'activation_function': 'softmax', 'test_loss': 572.2593994140625}
{'activation_function': 'tanh', 'test_loss': 304.16571044921875}

Hasil eksperimen jumlah epochs:
{'epochs': 1, 'test_loss': 666.0958862304688}
{'epochs': 10, 'test_loss': 340.1918640136719}
{'epochs': 25, 'test_loss': 56.